In [1]:
import os
import sys
import random
import pstats

from profile import Profile as PythonProfiler

import pandas as pd
import hippiehug

sys.path.append('..')

from claimchain import State, View, LocalParams
from claimchain.utils.profiling import Profiler

## Generating test data

This simulates `nb_readers` (200 by default) readers, each with own set of cryptographic keys. Every reader gets a random **32 byte** label associated with them, and **1 Kb** randomly generated claim about them.

The random capability matrix is generated: for each of the target readers _t_, `nb_caps_per_reader` (5 by default) other readers are randomly chosen. These are the ones that the reader _t_ will be able to access claims about, i.e. be included in _t_'s capability list.



In [2]:
def generate_test_data(nb_readers=200, nb_caps_per_reader=5):
    # size = {32, 1024} - len({"label_", "claim_"})
    labels = [b'label_' + os.urandom(32 - 6) for _ in range(nb_readers)]
    claims = [b'claim_' + os.urandom(1024 - 6) for _ in range(nb_readers)]

    params = [LocalParams.generate() for _ in range(nb_readers)]

    reader_graph = {}
    for reader_index in range(nb_readers):
        cap = random.sample(range(nb_readers), nb_caps_per_reader)
        reader_graph[reader_index] = cap

    return reader_graph, labels, claims, params

## Simulating the owner
The owner builds a state consisting of the claims generated above, with the access capabilities as generated above. The state is committed to the chain.

In [3]:
def commit_claims(reader_graph, labels, claims, params):
    owner_params = LocalParams.generate()
    with owner_params.as_default():
        state = State()
        
        # Add claims
        for label, claim in zip(labels, claims):
            state[label] = claim
        
        # Define the capability lists
        for reader_index, cap_indexes in reader_graph.items():
            reader_dh_pk = params[reader_index].dh.pk
            cap_labels = [labels[cap_index] for cap_index in cap_indexes]
            state.grant_access(reader_dh_pk, cap_labels)
        
        # Commit
        chain = hippiehug.Chain()
        state.commit(target_chain=chain)

        return chain, state

## Simulating the readers

In [4]:
def read_claims(chain, reader_graph, labels, claims, params):
    # Go over all of the readers
    for reader_index, caps_indexes in reader_graph.items():
        reader_params = params[reader_index]
        with reader_params.as_default():
            view = View(chain)
            # Retrieve each of the accessible labels
            for cap_index in caps_indexes:
                label = labels[cap_index]
                read_claim = view[labels[cap_index]]
                
                # Check the value matches original to be sure
                assert read_claim == claims[cap_index]

## Measuring timings

In [5]:
def simulate():
    reader_graph, labels, claims, params = generate_test_data()

    # Commit claims
    chain, state = commit_claims(reader_graph, labels, claims, params)
    # Simulate readers
    read_claims(chain, reader_graph, labels, claims, params)

### Low-level profiling

In [6]:
profiler = PythonProfiler()
profiler.runctx("simulate()", globals(), locals())
profiler.create_stats()

profiler.print_stats(sort='cumtime')

         1256576 function calls (1174676 primitive calls) in 6.408 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.408    6.408 profile:0(simulate())
        1    0.000    0.000    6.408    6.408 :0(exec)
        1    0.004    0.004    6.408    6.408 <string>:1(<module>)
        1    0.000    0.000    6.401    6.401 <ipython-input-5-8ff7cb198a40>:1(simulate)
        1    0.007    0.007    5.080    5.080 <ipython-input-4-1e1a01598d33>:1(read_claims)
4201/3201    0.024    0.000    5.023    0.002 profiling.py:34(wrapped)
     1000    0.006    0.000    3.889    0.004 state.py:196(__getitem__)
     1000    0.006    0.000    3.270    0.003 state.py:185(_lookup_claim)
     1000    0.028    0.000    3.211    0.003 core.py:67(decode_claim)
     1000    0.043    0.000    2.993    0.003 vrf.py:35(verify_vrf)
     1800    0.010    0.000    1.676    0.001 ec.py:245(from_binary)
     1800    1.655    0.

### High-level profiling

In [8]:
profiler = Profiler()
with profiler.as_default():
    simulate()
    stats = profiler.compute_stats()

In [9]:
df = pd.DataFrame(stats)
df = df.T
stat_cols = ['avg', 'max', 'min', 'std']
df[stat_cols] = df[stat_cols].apply(lambda value: value * 1000)  # milliseconds
df.sort_values(by='avg')

,avg,max,min,num,std
encode_capability,0.137249,0.252724,0.133514,1000.0,0.008858
_lookup_capability,0.280062,0.830889,0.256777,1000.0,0.047569
encode_claim,1.480722,2.094030,1.200199,200.0,0.135502
decode_claim,2.489942,4.513264,2.015591,1000.0,0.278126
_lookup_claim,2.506397,4.542351,2.028704,1000.0,0.280165
_build_tree,59.759855,59.759855,59.759855,1.0,NaN
